In [ ]:
!pip install scikit-learn -q
!pip install torchsummaryX wandb --quiet
!pip install tqdm --quiet

In [65]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchsummaryX import summary
import wandb
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [66]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

MPS backend is available!


In [67]:
def preprocess():
    df = pd.read_csv("merged_output2.csv")
    na_rows = df[df.isna().any(axis=1)]
    df = df.dropna()
    train, temp = train_test_split(df, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)

    return train, val, test
train, val, test = preprocess()

# NN MLP

In [68]:
class OccupancyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        return (torch.tensor(self.x.iloc[index].values, dtype=torch.float32),
                torch.tensor(self.y.iloc[index], dtype=torch.float32))


training = OccupancyDataset(train.iloc[:, 2:5],  train.iloc[:, -1])
validation = OccupancyDataset(val.iloc[:, 2:5],  val.iloc[:, -1])
test = OccupancyDataset(test.iloc[:, 2:5],  test.iloc[:, -1])

In [69]:
config = {
    'activations': 'GELU',
    'learning_rate': 0.001,
    'max_lr' : 0.006,
    'pct_start': 0.1,
    'optimizers': 'AdamW',
    'scheduler': 'OneCycleLR', #'ReduceLROnPlateau'
    'epochs': 25,
    'batch_size': 32,
    'weight_initialization': 'kaiming_normal', # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'dropout': 0.2
 }

In [70]:
train_loader = torch.utils.data.DataLoader(
    dataset     = training,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
)


val_loader = torch.utils.data.DataLoader(
    dataset     = validation,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

In [71]:
all = []
for i, data in enumerate(val_loader):
    sensor_data, target = data
    all.append(target)
    print(len(sensor_data))
    break

32


In [72]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers, dropout_rate):
        super().__init__()
        self.sequential = nn.ModuleList()

        #input layer
        self.sequential.append(nn.Linear(input_dim, hidden_dim))
        self.sequential.append(nn.ReLU())
        self.sequential.append(nn.BatchNorm1d(hidden_dim))

        #hidden
        for i in range(layers):
            self.sequential.append(nn.Linear(hidden_dim, hidden_dim))
            self.sequential.append(nn.ReLU())
            self.sequential.append(nn.BatchNorm1d(hidden_dim))

        #output layer
        self.sequential.append(nn.Linear(hidden_dim, output_dim))

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                if config["weight_initialization"] == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                elif config["weight_initialization"] == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                elif config["weight_initialization"] == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "uniform":
                    torch.nn.init.uniform_(m.weight)
                else:
                    raise ValueError("Invalid weight_initialization value")
                m.bias.data.fill_(0)
    def forward(self, x):
        for layer in self.sequential:
            x = layer(x)
        return x

In [140]:
model = MLP(input_dim=3, 
            hidden_dim=30, 
            output_dim=1, 
            layers = 20, 
            dropout_rate= config["dropout"]).to(device)
summary(model, sensor_data.to(device))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                     [3, 30]             [32, 30]                 0.12                 0.00
1_ReLU                             -             [32, 30]                    -                    -
2_BatchNorm1d                   [30]             [32, 30]                 0.06                 0.00
3_Linear                    [30, 30]             [32, 30]                 0.93                 0.00
4_ReLU                             -             [32, 30]                    -                    -
5_BatchNorm1d                   [30]             [32, 30]                 0.06                 0.00
6_Linear                    [30, 30]             [32, 30]                 0.93                 0.00
7_ReLU                             -             [32, 30]                    -                    -

In [74]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                       max_lr = config['max_lr'], 
                                                       pct_start = config['pct_start'], 
                                                       steps_per_epoch=len(train_loader),
                                                       anneal_strategy = 'cos',
                                                       epochs=config["epochs"]
                                                       )
criterion = torch.nn.MSELoss()


In [75]:
wandb.login(

    key="f449e5715dbf82026aae85dffabb14116b5aa142"

)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [76]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    tloss = 0.0
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, desc="Training")
    for i, (input, target) in enumerate(dataloader):
        optimizer.zero_grad()

        input = input.to(device)
        target = target.to(device).unsqueeze(1)
        #print(target.shape)

        logits = model(input)
        #print(logits.shape)
        loss = criterion(logits, target)
        tloss+=loss.item()

        loss.backward()
        optimizer.step()
        

        batch_bar.update()
        scheduler.step()
    batch_bar.close()
    tloss /= len(train_loader)
    return tloss


In [77]:
def eval(model, dataloader):
    model.eval()
    vloss = 0.0
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Validation')
    with torch.no_grad():
        for i, (input, target) in enumerate(dataloader):
            optimizer.zero_grad()

            input = input.to(device)
            target = target.to(device).unsqueeze(1)
            

            logits = model(input)
            loss = criterion(logits, target)
            vloss+=loss.item()

            batch_bar.update()
            #print("logits shape:", logits.shape)
            #print("target shape:", target.shape)

    batch_bar.close()
    vloss /= len(val_loader)
    return vloss

In [78]:
run = wandb.init(
    project="autonomous_project",  # Specify your project
    config = config,
    name = "run8"
)

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [83]:
loss = float("inf")
for epoch in range(config["epochs"]):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))
    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss   = train(model, train_loader, optimizer, criterion)
    val_loss       = eval(model, val_loader)

    print(f"Epoch {epoch}/{config['epochs']}", flush=True)
    print(f"\tTrain Loss {train_loss:.04f}\t Learning Rate {curr_lr:.07f}")
    print(f"\tVal Loss {val_loss:.04f}")

    wandb.log({'train_loss': train_loss,
               'valid_loss': val_loss, 'lr': curr_lr})
    if val_loss<loss:
        loss = val_loss
        print("saving")

        torch.save(model.state_dict(), "best_model")


Epoch 1/25


Training: 100%|██████████| 122/122 [00:02<00:00, 42.64it/s]
                                                  

Epoch 0/25


	Train Loss 10.5270	 Learning Rate 0.0002400
	Val Loss 7.1907
saving

Epoch 2/25


Training: 100%|██████████| 122/122 [00:02<00:00, 41.43it/s]
                                                  

...

Training: 100%|██████████| 122/122 [00:02<00:00, 42.26it/s]
                                                  

Epoch 23/25


	Train Loss 0.1554	 Learning Rate 0.0001153
	Val Loss 0.1515

Epoch 25/25


Training: 100%|██████████| 122/122 [00:02<00:00, 41.12it/s]
                                                            

Epoch 24/25
	Train Loss 0.1641	 Learning Rate 0.0000287
	Val Loss 0.1515


In [84]:
wandb.finish()

lr,▁▄▇████▇▇▇▆▆▅▅▄▄▃▃▃▂▂▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,3e-05
train_loss,0.16407
valid_loss,0.15147


In [129]:
import numpy as np
def test(model, test_loader):
    model.eval()
    test_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            inputs = batch[0].to(device)

            logits = model(inputs)
            test_predictions.append(np.round(logits.cpu().numpy()))

    return test_predictions

In [127]:
len(test_loader)

27

In [130]:
predictions = test(model, test_loader)

100%|██████████| 27/27 [00:00<00:00, 237.63it/s]


In [131]:
predictions[0]

array([[4.],
       [4.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [4.],
       [3.],
       [4.],
       [3.],
       [3.],
       [3.],
       [3.],
       [4.]], dtype=float32)

In [132]:
for batch_idx, (inputs, labels) in enumerate(test_loader):
    print(f"Batch {batch_idx}")
    print("Labels:", labels.numpy())

Batch 0
Labels: [3. 4. 3. 4. 4. 4. 4. 3. 3. 4. 4. 4. 3. 4. 3. 4. 3. 4. 3. 4. 3. 4. 2. 4.
 4. 3. 4. 3. 4. 3. 3. 4.]
Batch 1
Labels: [3. 4. 4. 3. 4. 4. 3. 3. 3. 4. 4. 3. 4. 3. 4. 3. 3. 4. 2. 4. 4. 4. 4. 4.
 4. 2. 4. 4. 4. 4. 4. 4.]
Batch 2
Labels: [3. 4. 4. 2. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 3. 4. 3. 4. 3. 3. 3. 4. 4. 4.
 2. 3. 3. 4. 3. 3. 3. 3.]
Batch 3
Labels: [4. 3. 3. 4. 3. 4. 4. 2. 4. 4. 3. 4. 3. 2. 3. 4. 3. 4. 4. 4. 4. 3. 4. 4.
 4. 3. 4. 3. 3. 3. 2. 4.]
Batch 4
Labels: [4. 4. 4. 4. 2. 3. 4. 4. 4. 3. 4. 3. 4. 3. 4. 3. 4. 3. 3. 3. 4. 3. 4. 4.
 3. 4. 4. 4. 3. 4. 4. 4.]
Batch 5
Labels: [3. 4. 3. 3. 4. 3. 4. 3. 4. 4. 4. 4. 4. 4. 2. 3. 3. 4. 4. 4. 4. 3. 4. 4.
 3. 2. 3. 3. 3. 3. 4. 4.]
Batch 6
Labels: [4. 4. 4. 4. 3. 3. 3. 4. 4. 3. 3. 3. 4. 3. 4. 4. 4. 4. 4. 3. 3. 4. 3. 3.
 3. 4. 4. 3. 3. 4. 4. 3.]
Batch 7
Labels: [3. 3. 2. 4. 4. 3. 3. 3. 4. 3. 3. 3. 4. 3. 2. 4. 4. 4. 4. 2. 3. 4. 4. 3.
 3. 3. 3. 4. 4. 4. 3. 3.]
Batch 8
Labels: [3. 4. 4. 4. 2. 4. 3. 4. 3. 4. 3. 4. 4. 4. 4. 4. 4. 3. 3. 3. 2. 

In [146]:
import torch
import numpy as np

# Load the model correctly
state_dict = torch.load('best_model')  # Load the saved state dict
model.load_state_dict(state_dict)

# Assuming 'model' is your trained model
model.eval()
with torch.no_grad():
    # Correctly pass the input as a tensor
    logits = model(torch.tensor([3.0, 4.0, 3.0]).unsqueeze(0).to(device))  # Make sure to cast to float tensor
    rounded_logits = np.round(logits.cpu().numpy())  # Round the logits

print(rounded_logits)


[[3.]]
